In [6]:
from collections import Counter
import pprint
import numpy as np
from tqdm import tqdm

In [8]:
f = open(f"./train_processed.txt", "r")
sentences = f.readlines()
f.close()
print(len(sentences))

sentences = [sentence.rstrip() for sentence in sentences if len(sentence) > 2]
sentences[27]

483449


'Coldwatch_NP0 1991_CRD'

In [9]:
word_tag_pairs = []
for sentence in sentences:
    for word_tag_pair in sentence.split(" "):
        if len(word_tag_pair.strip()) != 0:
            word_tag_pairs.append(word_tag_pair.strip())
print(len(word_tag_pairs))
print(word_tag_pairs[4])

8899819
by_PRP


In [10]:
for pair in word_tag_pairs:
    try:
        tag = pair.split("_")[1]
    except:
        print(pair)
        word_tag_pairs.remove(pair)

no
per


In [13]:
 for word_tag_pair in tqdm(word_tag_pairs):
        word = word_tag_pair.split("_")[0]
        tag = word_tag_pair.split("_")[1]
        if "_" in tag:
            word_tag_pairs.remove(word_tag_pair)
            t1 = tag.split("_")[0]
            t2 = tag.split("_")[1]
            word_tag_pairs.append(f"{word}_{t1}")
            word_tag_pairs.append(f"{word}_{t2}")
print(word_tag_pairs[4])
print(len(word_tag_pairs))

100%|██████████| 9007481/9007481 [00:07<00:00, 1156687.01it/s]

by_PRP
9007481


In [14]:
all_words = [pair.split('_')[0] for pair in word_tag_pairs]
all_tags = [pair.split('_')[1] for pair in word_tag_pairs]
word_freq = Counter(all_words).most_common() 
tag_freq = Counter(all_tags).most_common() 
train_words = [word[0] for word in word_freq] 
tags = [word[0] for word in tag_freq] 

In [15]:
print(word_tag_pairs[4])
print(all_words[4])
print(word_freq[10])
print(train_words[10])
print(all_tags[4])
print(tag_freq[10])
print(tags[1])
print(f"No. of unique words in train dataset = {len(train_words)}")
with open("./train_words.txt", "w+") as f:
    for word in train_words:
        f.write(f"{word}\n")
    f.close()

by_PRP
by
("'s", 69331)
's
PRP
('VVI', 212327)
AT0
No. of unique words in train dataset = 188189


In [16]:
tags_to_index = {}
for index in range(len(tag_freq)):
    tags_to_index[tag_freq[index][0]] = index
#print(tags_to_index)
tags_to_index["start"] = len(tags_to_index) #last tag is start tag
np.save("tag_to_index.npy", tags_to_index)

index_to_tags = {}
for index in range(len(tag_freq)):
    index_to_tags[index] = tag_freq[index][0]
index_to_tags[len(tags_to_index)] = "start" #last tag is start tag
np.save("index_to_tag.npy", index_to_tags)
print(index_to_tags)

{0: 'NN1', 1: 'AT0', 2: 'PRP', 3: 'AJ0', 4: 'NN2', 5: 'NP0', 6: 'AV0', 7: 'PNP', 8: 'CJC', 9: 'PRF', 10: 'VVI', 11: 'VVN', 12: 'DT0', 13: 'VVD', 14: 'CRD', 15: 'TO0', 16: 'DPS', 17: 'VVG', 18: 'VM0', 19: 'VBD', 20: 'CJS', 21: 'VBZ', 22: 'VVB', 23: 'VVZ', 24: 'CJT', 25: 'AVP', 26: 'POS', 27: 'XX0', 28: 'VBI', 29: 'DTQ', 30: 'NN0', 31: 'VBB', 32: 'VHD', 33: 'ORD', 34: 'NN1-VVB', 35: 'VHZ', 36: 'AJ0-NN1', 37: 'VHB', 38: 'PNI', 39: 'NP0-NN1', 40: 'VBN', 41: 'PNQ', 42: 'EX0', 43: 'AVQ', 44: 'AJC', 45: 'VHI', 46: 'PRP-AVP', 47: 'NN1-AJ0', 48: 'VVN-VVD', 49: 'VVD-VVN', 50: 'VVB-NN1', 51: 'PNX', 52: 'UNC', 53: 'AJS', 54: 'NN1-NP0', 55: 'VDB', 56: 'VDD', 57: 'VVG-AJ0', 58: 'VBG', 59: 'NN2-VVZ', 60: 'VVG-NN1', 61: 'VVN-AJ0', 62: 'ITJ', 63: 'VDI', 64: 'AVP-PRP', 65: 'AV0-AJ0', 66: 'CJT-DT0', 67: 'ZZ0', 68: 'VDZ', 69: 'NN1-VVG', 70: 'AJ0-AV0', 71: 'VVZ-NN2', 72: 'AJ0-VVG', 73: 'PRP-CJS', 74: 'CJS-AVQ', 75: 'AJ0-VVN', 76: 'CJS-PRP', 77: 'VHG', 78: 'DT0-CJT', 79: 'VDN', 80: 'VHN', 81: 'AJ0-VVD', 82:

In [17]:
em_prob_of_words_for_each_tag = {}
for word_tag_pair in tqdm(word_tag_pairs):
    word = word_tag_pair.split("_")[0]
    tag = word_tag_pair.split("_")[1]
    if "-" not in tag:
        if tag not in em_prob_of_words_for_each_tag:
            em_prob_of_words_for_each_tag[tag] = {}
            
        if word not in em_prob_of_words_for_each_tag[tag]:
            em_prob_of_words_for_each_tag[tag][word] = 0
        em_prob_of_words_for_each_tag[tag][word] += 1
    else:
        tag1 = tag.split("-")[0]
        tag2 = tag.split("-")[1]
        if tag1 not in em_prob_of_words_for_each_tag:
            em_prob_of_words_for_each_tag[tag1] = {}
        if tag2 not in em_prob_of_words_for_each_tag:
            em_prob_of_words_for_each_tag[tag2] = {}

        if word not in em_prob_of_words_for_each_tag.get(tag1):
            em_prob_of_words_for_each_tag[tag1][word] = 0
        em_prob_of_words_for_each_tag[tag1][word] += 1
        if word not in em_prob_of_words_for_each_tag.get(tag2):
            em_prob_of_words_for_each_tag[tag2][word] = 0
        em_prob_of_words_for_each_tag[tag2][word] += 1

print(em_prob_of_words_for_each_tag["VVI"]["make"])

100%|██████████| 9007481/9007481 [00:10<00:00, 863641.51it/s]

5377


In [18]:
for tag in em_prob_of_words_for_each_tag:
    total_words_forthis_tag = 0
    
    for word in em_prob_of_words_for_each_tag.get(tag):
        total_words_forthis_tag += 1

    for word in em_prob_of_words_for_each_tag.get(tag):
        em_prob_of_words_for_each_tag[tag][word]

In [19]:
np.save("word_given_tag.npy", em_prob_of_words_for_each_tag)

In [20]:
loaded_freq_of_words_for_each_tag = np.load("word_given_tag.npy",allow_pickle=True).item()

In [21]:
print(f"the tags for be checked: VVI")
print(loaded_freq_of_words_for_each_tag.get("VVI"))

the tags for be checked: VVI
{'miss': 212, 'see': 4646, 'attend': 215, 'form': 500, 'update': 23, 'continue': 914, 'look': 1757, 'derive': 35, 'keep': 1763, 'like': 1433, 'welcome': 100, 'Cut': 1, 'receive': 418, 'come': 2303, 'need': 980, 'recruit': 65, 'support': 540, 'go': 4480, 'assist': 136, 'reflect': 199, 'reach': 584, 'follow': 537, 'ensure': 812, 'participate': 107, 'subsidise': 20, 'retain': 212, 'cover': 397, 'apply': 366, 'comment': 110, 'discuss': 323, 'help': 1855, 'put': 1355, 'use': 1373, 'provide': 1329, 'enlist': 12, 'thank': 68, 'hear': 805, 'persuade': 283, 'care': 307, 're-launch': 2, 'refer': 106, 'feature': 49, 'nominate': 17, 'join': 678, 'share': 303, 'benefit': 217, 'publicise': 13, 'highlight': 30, 'suggest': 340, 'ask': 749, 'vote': 293, 'talk': 687, 'delay': 74, 'generate': 98, 'develop': 578, 'set': 619, 'increase': 550, 'schedule': 8, 'make': 5377, 'guide': 58, 'charge': 78, 'get': 4782, 'take': 4461, 'buy': 979, 'cost': 263, 'act': 347, 'effect': 32, 'en

In [22]:
transition_matrix = np.zeros([len(tags_to_index),len(tags_to_index)])

In [23]:
for sentence in sentences:
    for word_tag_pair_idx in range(len(sentence.split(" "))):
        word_tag_pair = sentence.split(" ")[word_tag_pair_idx]
        if len(word_tag_pair.strip()) != 0:
            try:
                tag = pair.split("_")[1]
                if word_tag_pair_idx==0:
                    prev_tag = "start"
                else:
                    prev_tag = sentence.split(" ")[word_tag_pair_idx-1].split("_")[1]
                curr_tag = word_tag_pair.split("_")[1]
                transition_matrix[tags_to_index[prev_tag]][tags_to_index[curr_tag]] += 1
            except:
                continue

for idx in range(len(tags_to_index)):
    total = 0
    for idx2 in range(len(tags_to_index)):
        total += transition_matrix[idx][idx2]
    for idx2 in range(len(tags)):
        transition_matrix[idx][idx2] = transition_matrix[idx][idx2]/total

In [24]:
np.save("Trans_Matrix.npy",transition_matrix)

In [25]:
print(transition_matrix[56])

[5.31806095e-03 4.07036204e-02 3.23174473e-02 3.98854571e-03
 4.09081612e-03 1.61587237e-02 4.20331356e-02 1.06770301e-01
 8.18163224e-03 3.06811209e-04 6.92370628e-02 0.00000000e+00
 2.62834936e-02 9.20433627e-04 2.96584169e-03 8.18163224e-04
 1.11474739e-02 9.20433627e-04 5.11352015e-04 3.88627531e-03
 6.23849458e-03 1.02270403e-04 6.13622418e-04 1.02270403e-04
 1.32951524e-03 0.00000000e+00 0.00000000e+00 5.89588873e-01
 1.02270403e-04 3.17038249e-03 5.11352015e-04 1.02270403e-04
 1.02270403e-04 1.02270403e-03 0.00000000e+00 0.00000000e+00
 4.09081612e-04 0.00000000e+00 1.13520147e-02 1.02270403e-04
 0.00000000e+00 0.00000000e+00 3.06811209e-04 2.04540806e-04
 2.04540806e-04 4.39762733e-03 1.02270403e-04 0.00000000e+00
 2.04540806e-04 1.02270403e-04 0.00000000e+00 5.11352015e-04
 1.02270403e-04 1.02270403e-04 0.00000000e+00 1.02270403e-04
 3.06811209e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.02270403e-04 0.00000000e+00 2.04540806e-04 6.13622418e-04
 0.00000000e+00 4.090816

In [26]:
#np.save("tag_to_index.npy",tag_to_index)

NameError: name 'tag_to_index' is not defined